## MSDS 432 - Programming Assignment #9 Dan Avni

The purpose of this programming assignment is to explore Dynamic Programming, a general method for algorithmically solving complex problems by breaking the problem to simpler subproblems, solving the subproblems individually, and storing the solution in memory for later assembly toward the overall solution.

The Dynamic Programming approach has many useful applications in domains such as engineering, search, and economics. We will be looking at the 'Edit Distance' problem: given $n$ strings, find the minimal number of operation to derive a destination string from a source string using an insert, modify, or delete operation. The edit distance algorithm is useful for applications such as spelling and data entry checks.

The worst-case complexity of an optimal solution is still exponential: $O(\sum_{i,j=1}^{n}len(n_i)*len(n_j))$ 

We demonstrate below that a Dynamic Programming implementation performs markedly better than a naive brute-force approach by observing the runtime edit distance performance of all possible word pairs in the cannoical sentence "The quick brown Fox jumped over The lazy dog", one of the shortest English langauge sentences that includes all the letters of the alphabet. We verify that both algorithm produce the same edit distance by producing a table of all possible 144 edit distance combinations.

See below diagram for runtime distribution of both algorithms as err bar plots: 

<img width="45%" height="45%" src="https://raw.githubusercontent.com/dannyavni/MSDS_432/master/runtime8.png"/>

In [2]:
#import required packages
import numpy as np
import pandas as pd
import time

In [3]:
#adapted from: https://www.sanfoundry.com/dynamic-programming-solutions-edit-distance-problem/

def min_edit_distance_brute(src, dest, len1, len2):
    #if we have already attained the dest string, we can simply remove the left out characters in src string    
    if len2==0:
        return len1
 
    #if the entire src string is process, we can simply insert all the remaining characters of dest string
    if len1==0:
        return len2
 
    #if current character of the src and dest is same, no operation is required to be done
    if src[len1-1]==dest[len2-1]:
        return min_edit_distance_brute(src, dest, len1-1, len2-1)
    #if current character is different for src and dest string
    else:
        #if the required character of dest string is inserted in src string
        x=1+min_edit_distance_brute(src,dest,len1,len2-1)
 
        #if the current character of src is removed
        y=1+min_edit_distance_brute(src,dest,len1-1,len2)
 
        #if the current character of the src string is replaced with the current character of the dest string
        z=1+min_edit_distance_brute(src,dest,len1-1,len2-1)
 
        #now, choose the operation which require minimum number of overall operations out of these
        return min(x,min(y,z))

In [4]:
#adapted from: https://www.sanfoundry.com/dynamic-programming-solutions-edit-distance-problem/
def min_edit_distance(src, dest):
    len1, len2 = len(src), len(dest)
 
    #create a matrix of order (len1+1)*(len2+1) to memoize values
    edit = np.empty([len1+1,len2+1], dtype=int)
 
    #edit[i][j]=minimum number of edit operations required to transform src[0....(i-1)] to dest[0...(j-1)]
 
    for i in range(0, len1+1):
        edit[i][0]=i    #min operations required to transform src[0...i-1] to empty dest string
 
    for j in range(0, len2+1):
        edit[0][j]=j   #min operations required to transform empty src to dest[0...j-1]
 
    #now, start filling the matrix row wise
    for i in range(1, len1+1):
        for j in range(1, len2+1):
            #if current character of both strings match
            if(src[i-1]==dest[j-1]):            
                edit[i][j]=edit[i-1][j-1]
            #mismatch
            else:
                #try applying all operations and choose the one which costs minimum
                x=1+edit[i-1][j];    #delete 
                y=1+edit[i][j-1];    #insert
                z=1+edit[i-1][j-1];  #replace
                edit[i][j]=min(x,min(y,z))
                
    #now, return the final value 
    return edit[len1][len2];

In [5]:
#unit test for DP 
min_edit_distance("danny", "mika")

5

In [6]:
#unit test for brute force
min_edit_distance_brute("mika", "danny", 4, 5)

5

In [7]:
sentence = "the quick brown fox jumped over the lazy dog"

In [8]:
words = sentence.split(" ")

In [9]:
test_results_col_names = ['method', 'word1', 'word2', 'edit_distance', 'runtime_ms']
test_results           = pd.DataFrame(columns = test_results_col_names) 

In [10]:
for i in range(0, len(words)):
    for j in range(0, len(words)):
        if (i != j):
            word1 = words[i]
            word2 = words[j]
            
            start   = time.time()
            dist = min_edit_distance(word1, word2)
            runtime_ms = ((time.time() - start) * 1000)
            test_results.loc[len(test_results)] = ['Dynamic Programming Edit Distance', word1, word2, dist, runtime_ms]
            
            start   = time.time()
            dist = min_edit_distance_brute(word1, word2, len(word1), len(word2))
            runtime_ms = ((time.time() - start) * 1000)
            test_results.loc[len(test_results)] = ['Brute Force Edit Distance', word1, word2, dist, runtime_ms]


In [11]:
test_results

,method,word1,word2,edit_distance,runtime_ms
0,Dynamic Programming Edit Distance,the,quick,5,0.094652
1,Brute Force Edit Distance,the,quick,5,0.229836
2,Dynamic Programming Edit Distance,the,brown,5,0.066519
3,Brute Force Edit Distance,the,brown,5,0.124454
4,Dynamic Programming Edit Distance,the,fox,3,0.342131
5,Brute Force Edit Distance,the,fox,3,0.034809
6,Dynamic Programming Edit Distance,the,jumped,5,0.061750
7,Brute Force Edit Distance,the,jumped,5,0.162601
8,Dynamic Programming Edit Distance,the,over,3,0.046253
9,Brute Force Edit Distance,the,over,3,0.048637


In [12]:
dp_mean = test_results.loc[test_results['method']=='Dynamic Programming Edit Distance', 'runtime_ms'].mean()

In [13]:
dp_mean

0.06790293587578668

In [14]:
bf_mean = test_results.loc[test_results['method']=='Brute Force Edit Distance', 'runtime_ms'].mean()

In [15]:
bf_mean

0.21406345897250706

In [16]:
import seaborn as sns

In [17]:
sns.set(rc={'figure.figsize':(8, 8)})
sns.boxplot(x="method", y="runtime_ms", data=test_results, palette="Set2")